## SETUP 

In [1]:
#import libraries
import pandas as pd
import sqlite3

In [4]:
#df = pd.DataFrame(#data here :/)

df = pd.DataFrame({'name': ['Juan', 'Victoria', 'Mary'], \
'age': [23, 34, 43], 'city': ['Miami', 'Buenos Aries', 'Santiago']})


In [5]:
df

,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [7]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_tutorial.db')

In [11]:
df.to_sql('people', cnn)

In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///jupyter_sql_tutorial.db

In [17]:
%%sql
SELECT *
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


index,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [18]:
%%sql
SELECT count(*)
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


count(*)
3


In [19]:
%%sql
SELECT sum(age) as 'age_sum'
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


age_sum
100


## Parameters walkthrough

In [20]:
#create dummy dataframe
df = pd.DataFrame({'transaction_id': ['9', '8', '7', '6', '5', '4', '3'], \
                  'user_id': ['rafa', 'roy', 'kenny', 'brendan', 'jurgen', 'roy', 'roy'],\
                  'transaction_date': ['2021-12-21', '2020-12-21', '2019-12-21',\
                                       '2018-11-21', '2017-10-21', '2019-03-02', '2010-01-01'],\
                  'amount': ['10', '15', '20', '24', '25', '31', '42']})

In [21]:
df

,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [22]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('dummy.db')

In [29]:
df.to_sql('managers1', cnn)

In [30]:
%reload_ext sql

In [31]:
%sql sqlite:///dummy.db

In [32]:
%%sql
SELECT *
FROM managers1

 * sqlite:///dummy.db
Done.


index,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [34]:
%%sql
SELECT sum(amount) as 'spend_sum'
FROM managers1

 * sqlite:///dummy.db
Done.


spend_sum
167


In [35]:
%%sql
SELECT 
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
FROM
    managers1
WHERE
    user_id = 'roy'
    and transaction_date = '2019-03-02'

 * sqlite:///dummy.db
Done.


user_id,num_transactions,total_amount
roy,1,31


## Pandas solution

ovechal01, ovi8

1228 * 11,


In [1]:
#import libraries
from datetime import timedelta
from dateutil.parser import parse
import pandas as pd
import sqlite3
import time

In [2]:
stat_handle = input('stat handle:')
stat_sheet = f'./data/{stat_handle}_stats.csv'
stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

stat handle:ovechal01


In [3]:
stat_df

,date,goals,assists,+/-,pim,shots,shot%,TOI,hits,blocks,faceoff %
0,2013-01-19 19:00:00-05:00,0,0,0,2,4,0.0,23:35,2,0,NaN
1,2013-01-22 19:00:00-05:00,0,1,0,0,4,0.0,23:59,4,0,NaN
2,2013-01-24 19:00:00-05:00,0,0,-1,2,3,0.0,22:26,1,0,NaN
3,2013-01-25 19:00:00-05:00,0,0,0,0,2,0.0,22:29,1,0,NaN
4,2013-01-27 15:00:00-05:00,1,0,-1,0,5,20.0,18:29,3,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
675,2021-12-29 19:00:00-05:00,0,1,1,0,4,0.0,26:01,0,0,NaN
676,2021-12-31 19:30:00-05:00,2,0,2,0,7,28.6,19:54,1,0,NaN
677,2022-01-02 15:00:00-05:00,0,0,-2,0,3,0.0,22:56,4,0,100.0
678,2022-01-07 20:00:00-05:00,0,0,-3,0,3,0.0,20:55,1,0,NaN


In [4]:
def find_dates(player):
    dates = []
    stat_handle = player
    stat_sheet = f'./data/{stat_handle}_stats.csv'
    stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)
    for row in stat_df.iterrows():
        values = row[1]
        pd_date = parse(values[0])
        
        #time delta to 1 day prior, 
        #worth investigating the effect of using time delta to travel to midnight on gameday
        start_date = pd_date - timedelta(days = 1)
        time_range = (start_date, pd_date)
        #append to dates list
        dates.append(time_range)
    return dates
        
dates = find_dates('ovechal01')
for date in dates:
    print(date[0], '**', date[1], '\n\n')

2013-01-18 19:00:00-05:00 ** 2013-01-19 19:00:00-05:00 


2013-01-21 19:00:00-05:00 ** 2013-01-22 19:00:00-05:00 


2013-01-23 19:00:00-05:00 ** 2013-01-24 19:00:00-05:00 


2013-01-24 19:00:00-05:00 ** 2013-01-25 19:00:00-05:00 


2013-01-26 15:00:00-05:00 ** 2013-01-27 15:00:00-05:00 


2013-01-28 19:30:00-05:00 ** 2013-01-29 19:30:00-05:00 


2013-01-30 19:00:00-05:00 ** 2013-01-31 19:00:00-05:00 


2013-01-31 19:00:00-05:00 ** 2013-02-01 19:00:00-05:00 


2013-02-03 00:30:00-05:00 ** 2013-02-04 00:30:00-05:00 


2013-02-04 19:00:00-05:00 ** 2013-02-05 19:00:00-05:00 


2013-02-06 19:00:00-05:00 ** 2013-02-07 19:00:00-05:00 


2013-02-08 19:00:00-05:00 ** 2013-02-09 19:00:00-05:00 


2013-02-11 19:30:00-05:00 ** 2013-02-12 19:30:00-05:00 


2013-02-13 19:30:00-05:00 ** 2013-02-14 19:30:00-05:00 


2013-02-16 18:00:00-05:00 ** 2013-02-17 18:00:00-05:00 


2013-02-20 19:00:00-05:00 ** 2013-02-21 19:00:00-05:00 


2013-02-23 00:00:00-05:00 ** 2013-02-24 00:00:00-05:00 


2013-02-25 19:

2015-02-14 20:00:00-05:00 ** 2015-02-15 20:00:00-05:00 


2015-02-16 19:00:00-05:00 ** 2015-02-17 19:00:00-05:00 


2015-02-18 19:00:00-05:00 ** 2015-02-19 19:00:00-05:00 


2015-02-21 00:30:00-05:00 ** 2015-02-22 00:30:00-05:00 


2015-02-22 00:30:00-05:00 ** 2015-02-23 00:30:00-05:00 


2015-02-24 20:00:00-05:00 ** 2015-02-25 20:00:00-05:00 


2015-02-26 19:00:00-05:00 ** 2015-02-27 19:00:00-05:00 


2015-02-28 19:00:00-05:00 ** 2015-03-01 19:00:00-05:00 


2015-03-02 19:00:00-05:00 ** 2015-03-03 19:00:00-05:00 


2015-03-06 19:00:00-05:00 ** 2015-03-07 19:00:00-05:00 


2015-03-10 20:00:00-04:00 ** 2015-03-11 20:00:00-04:00 


2015-03-12 19:00:00-04:00 ** 2015-03-13 19:00:00-04:00 


2015-03-14 19:30:00-04:00 ** 2015-03-15 19:30:00-04:00 


2015-03-15 19:00:00-04:00 ** 2015-03-16 19:00:00-04:00 


2015-03-18 20:00:00-04:00 ** 2015-03-19 20:00:00-04:00 


2015-03-20 19:00:00-04:00 ** 2015-03-21 19:00:00-04:00 


2015-03-25 19:00:00-04:00 ** 2015-03-26 19:00:00-04:00 


2015-03-28 00:

2017-10-12 19:00:00-04:00 ** 2017-10-13 19:00:00-04:00 


2017-10-13 19:00:00-04:00 ** 2017-10-14 19:00:00-04:00 


2017-10-16 19:00:00-04:00 ** 2017-10-17 19:00:00-04:00 


2017-10-19 19:30:00-04:00 ** 2017-10-20 19:30:00-04:00 


2017-10-20 19:30:00-04:00 ** 2017-10-21 19:30:00-04:00 


2017-10-25 22:00:00-04:00 ** 2017-10-26 22:00:00-04:00 


2017-10-27 22:00:00-04:00 ** 2017-10-28 22:00:00-04:00 


2017-10-28 21:00:00-04:00 ** 2017-10-29 21:00:00-04:00 


2017-11-01 19:00:00-04:00 ** 2017-11-02 19:00:00-04:00 


2017-11-03 19:00:00-04:00 ** 2017-11-04 19:00:00-04:00 


2017-11-05 19:00:00-05:00 ** 2017-11-06 19:00:00-05:00 


2017-11-06 19:00:00-05:00 ** 2017-11-07 19:00:00-05:00 


2017-11-09 19:00:00-05:00 ** 2017-11-10 19:00:00-05:00 


2017-11-11 19:00:00-05:00 ** 2017-11-12 19:00:00-05:00 


2017-11-13 20:00:00-05:00 ** 2017-11-14 20:00:00-05:00 


2017-11-15 21:00:00-05:00 ** 2017-11-16 21:00:00-05:00 


2017-11-17 19:30:00-05:00 ** 2017-11-18 19:30:00-05:00 


2017-11-19 19:

2019-03-05 19:30:00-05:00 ** 2019-03-06 19:30:00-05:00 


2019-03-07 19:00:00-05:00 ** 2019-03-08 19:00:00-05:00 


2019-03-09 19:00:00-04:00 ** 2019-03-10 19:00:00-04:00 


2019-03-11 19:00:00-04:00 ** 2019-03-12 19:00:00-04:00 


2019-03-13 19:00:00-04:00 ** 2019-03-14 19:00:00-04:00 


2019-03-15 19:00:00-04:00 ** 2019-03-16 19:00:00-04:00 


2019-03-18 19:30:00-04:00 ** 2019-03-19 19:30:00-04:00 


2019-03-19 19:30:00-04:00 ** 2019-03-20 19:30:00-04:00 


2019-03-21 19:00:00-04:00 ** 2019-03-22 19:00:00-04:00 


2019-03-24 00:30:00-04:00 ** 2019-03-25 00:30:00-04:00 


2019-03-25 19:00:00-04:00 ** 2019-03-26 19:00:00-04:00 


2019-03-27 19:00:00-04:00 ** 2019-03-28 19:00:00-04:00 


2019-03-29 19:00:00-04:00 ** 2019-03-30 19:00:00-04:00 


2019-03-31 19:00:00-04:00 ** 2019-04-01 19:00:00-04:00 


2019-04-03 19:00:00-04:00 ** 2019-04-04 19:00:00-04:00 


2019-04-05 19:00:00-04:00 ** 2019-04-06 19:00:00-04:00 


2019-10-01 20:00:00-04:00 ** 2019-10-02 20:00:00-04:00 


2019-10-03 19:

2021-12-30 19:30:00-05:00 ** 2021-12-31 19:30:00-05:00 


2022-01-01 15:00:00-05:00 ** 2022-01-02 15:00:00-05:00 


2022-01-06 20:00:00-05:00 ** 2022-01-07 20:00:00-05:00 


2022-01-07 20:00:00-05:00 ** 2022-01-08 20:00:00-05:00 




In [5]:
stat_handle = input('twitter handle:')
stat_sheet = f'./data/{stat_handle}.csv'
twitter_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

twitter handle:ovi8


In [6]:
for row in twitter_df['date time']:
    print(row)

2021-12-02 18:19:51-05:00
2021-11-20 20:44:54-05:00
2021-11-16 04:04:50-05:00
2021-11-04 13:16:26-04:00
2021-10-28 22:15:31-04:00
2021-10-12 22:41:50-04:00
2021-08-07 19:00:04-04:00
2021-07-30 17:38:23-04:00
2021-07-27 17:01:06-04:00
2021-03-17 22:47:01-04:00
2021-03-04 23:43:43-05:00
2020-12-17 21:27:52-05:00
2020-12-03 18:31:04-05:00
2020-10-16 16:33:52-04:00
2020-10-08 17:29:38-04:00
2020-10-02 16:12:24-04:00
2020-09-03 14:11:57-04:00
2020-08-24 17:54:06-04:00
2020-06-10 00:11:30-04:00
2020-06-02 01:58:22-04:00
2020-04-30 17:32:52-04:00
2020-04-23 02:32:56-04:00
2020-04-22 01:53:32-04:00
2020-04-19 23:35:52-04:00
2020-04-07 21:21:04-04:00
2020-01-26 23:04:19-05:00
2020-01-14 04:27:02-05:00
2020-01-11 00:15:14-05:00
2019-12-22 18:18:40-05:00
2019-12-13 16:45:43-05:00
2019-11-29 15:38:54-05:00
2019-11-22 21:29:21-05:00
2019-11-21 21:18:27-05:00
2019-11-20 02:20:09-05:00
2019-10-30 20:34:39-04:00
2019-10-11 21:21:43-04:00
2019-10-10 05:02:29-04:00
2019-10-08 02:16:31-04:00
2019-10-01 1

In [7]:
from datetime import timedelta

tempo = pd.to_datetime('2009-01-21 14:51:20-05:00')
temp = tempo - timedelta(hours=1)
print(temp)

2009-01-21 13:51:20-05:00


In [8]:
start_dates = []
end_dates = []
gm_dates = stat_df.iloc[:,0]
for gm in gm_dates:
    try:
        gm = pd.to_datetime(gm)
        start_dates.append(gm)
        gm = gm - timedelta(days=1)
        end_dates.append(gm)
        
    except:
        continue

#game_windows = pd.DataFrame(data={'start_date':[x for x in start_dates],'end_date': [y for y in end_dates]})
#gw1 = list(pd.date_range(start = (x for x in start_dates), end= (y for y in end_dates)))
for x, y in zip(start_dates, end_dates):
    c = 0
    c+= 1
    with open('ovechkin_dates.csv', 'w', encoding='utf-8'):
        twitter_handle = 'ovi8'
        tweet_sheet = f'./data/{twitter_handle}.csv'
        twitter_df = pd.read_csv(tweet_sheet, sep= '\t', header= 0, index_col= None)
        #print(x, y)
        time = pd.date_range(start = (x), end= (y), freq = 'M')

        dates = find_dates('ovechal01')
        for date in dates:
            start = date[0]
            end = date[1]
            for row in twitter_df['date time']:
                row = parse(row)
                if row >= start and row <= end:
                    #add point to new DB
                    print(f'point found: {row}')


point found: 2013-03-07 17:22:54-05:00
point found: 2013-03-07 17:20:57-05:00
point found: 2013-03-14 02:57:16-04:00
point found: 2013-03-14 00:04:50-04:00
point found: 2013-03-14 00:00:46-04:00
point found: 2013-03-22 18:07:31-04:00
point found: 2013-03-22 18:04:13-04:00
point found: 2013-03-22 18:03:07-04:00
point found: 2013-03-24 12:54:47-04:00
point found: 2013-03-24 01:34:46-04:00
point found: 2013-03-24 01:30:36-04:00
point found: 2013-03-24 00:03:45-04:00
point found: 2013-03-23 19:03:04-04:00
point found: 2013-03-31 03:18:19-04:00
point found: 2013-04-04 15:48:15-04:00
point found: 2013-04-03 21:14:43-04:00
point found: 2013-04-05 23:35:23-04:00
point found: 2013-04-05 23:34:04-04:00
point found: 2013-04-07 15:00:57-04:00
point found: 2013-04-10 21:16:38-04:00
point found: 2013-04-10 20:39:59-04:00
point found: 2013-04-10 20:17:58-04:00
point found: 2013-04-17 19:46:44-04:00
point found: 2013-04-17 19:45:26-04:00
point found: 2013-04-17 19:44:16-04:00
point found: 2013-04-17 1

point found: 2020-01-11 00:15:14-05:00
point found: 2021-03-04 23:43:43-05:00
point found: 2021-10-12 22:41:50-04:00
point found: 2021-10-28 22:15:31-04:00
point found: 2021-11-04 13:16:26-04:00
point found: 2021-11-16 04:04:50-05:00
point found: 2021-11-20 20:44:54-05:00
point found: 2021-12-02 18:19:51-05:00
point found: 2013-03-07 17:22:54-05:00
point found: 2013-03-07 17:20:57-05:00
point found: 2013-03-14 02:57:16-04:00
point found: 2013-03-14 00:04:50-04:00
point found: 2013-03-14 00:00:46-04:00
point found: 2013-03-22 18:07:31-04:00
point found: 2013-03-22 18:04:13-04:00
point found: 2013-03-22 18:03:07-04:00
point found: 2013-03-24 12:54:47-04:00
point found: 2013-03-24 01:34:46-04:00
point found: 2013-03-24 01:30:36-04:00
point found: 2013-03-24 00:03:45-04:00
point found: 2013-03-23 19:03:04-04:00
point found: 2013-03-31 03:18:19-04:00
point found: 2013-04-04 15:48:15-04:00
point found: 2013-04-03 21:14:43-04:00
point found: 2013-04-05 23:35:23-04:00
point found: 2013-04-05 2

point found: 2018-04-04 22:52:00-04:00
point found: 2018-10-31 22:45:29-04:00
point found: 2019-02-14 02:13:55-05:00
point found: 2019-10-08 02:16:31-04:00
point found: 2019-10-10 05:02:29-04:00
point found: 2019-10-11 21:21:43-04:00
point found: 2019-11-20 02:20:09-05:00
point found: 2019-11-29 15:38:54-05:00
point found: 2020-01-11 00:15:14-05:00
point found: 2021-03-04 23:43:43-05:00
point found: 2021-10-12 22:41:50-04:00
point found: 2021-10-28 22:15:31-04:00
point found: 2021-11-04 13:16:26-04:00
point found: 2021-11-16 04:04:50-05:00
point found: 2021-11-20 20:44:54-05:00
point found: 2021-12-02 18:19:51-05:00
point found: 2013-03-07 17:22:54-05:00
point found: 2013-03-07 17:20:57-05:00
point found: 2013-03-14 02:57:16-04:00
point found: 2013-03-14 00:04:50-04:00
point found: 2013-03-14 00:00:46-04:00
point found: 2013-03-22 18:07:31-04:00
point found: 2013-03-22 18:04:13-04:00
point found: 2013-03-22 18:03:07-04:00
point found: 2013-03-24 12:54:47-04:00
point found: 2013-03-24 0

point found: 2017-02-05 20:29:56-05:00
point found: 2017-02-11 02:43:44-05:00
point found: 2017-03-03 22:15:25-05:00
point found: 2017-10-12 22:49:28-04:00
point found: 2017-10-13 20:34:24-04:00
point found: 2017-10-20 16:09:14-04:00
point found: 2017-12-28 17:28:31-05:00
point found: 2018-03-12 13:56:01-04:00
point found: 2018-04-04 22:52:00-04:00
point found: 2018-10-31 22:45:29-04:00
point found: 2019-02-14 02:13:55-05:00
point found: 2019-10-08 02:16:31-04:00
point found: 2019-10-10 05:02:29-04:00
point found: 2019-10-11 21:21:43-04:00
point found: 2019-11-20 02:20:09-05:00
point found: 2019-11-29 15:38:54-05:00
point found: 2020-01-11 00:15:14-05:00
point found: 2021-03-04 23:43:43-05:00
point found: 2021-10-12 22:41:50-04:00
point found: 2021-10-28 22:15:31-04:00
point found: 2021-11-04 13:16:26-04:00
point found: 2021-11-16 04:04:50-05:00
point found: 2021-11-20 20:44:54-05:00
point found: 2021-12-02 18:19:51-05:00
point found: 2013-03-07 17:22:54-05:00
point found: 2013-03-07 1

point found: 2014-04-12 19:21:46-04:00
point found: 2014-10-28 22:19:08-04:00
point found: 2015-01-09 21:46:27-05:00
point found: 2015-04-03 20:38:08-04:00
point found: 2015-12-16 17:10:25-05:00
point found: 2016-01-15 20:55:46-05:00
point found: 2016-12-09 18:24:57-05:00
point found: 2016-12-31 02:06:36-05:00
point found: 2017-02-05 20:29:56-05:00
point found: 2017-02-11 02:43:44-05:00
point found: 2017-03-03 22:15:25-05:00
point found: 2017-10-12 22:49:28-04:00
point found: 2017-10-13 20:34:24-04:00
point found: 2017-10-20 16:09:14-04:00
point found: 2017-12-28 17:28:31-05:00
point found: 2018-03-12 13:56:01-04:00
point found: 2018-04-04 22:52:00-04:00
point found: 2018-10-31 22:45:29-04:00
point found: 2019-02-14 02:13:55-05:00
point found: 2019-10-08 02:16:31-04:00
point found: 2019-10-10 05:02:29-04:00
point found: 2019-10-11 21:21:43-04:00
point found: 2019-11-20 02:20:09-05:00
point found: 2019-11-29 15:38:54-05:00
point found: 2020-01-11 00:15:14-05:00
point found: 2021-03-04 2

KeyboardInterrupt: 

In [ ]:
with open('ovechkin_dates.csv', 'w', encoding='utf-8'):
    point_list = []
    twitter_handle = 'ovi8'
    tweet_sheet = f'./data/{twitter_handle}.csv'
    twitter_df = pd.read_csv(tweet_sheet, sep= '\t', header= 0, index_col= None)
    time = pd.date_range(start = (x), end= (y), freq = 'M')

    dates = find_dates('ovechal01')
    for date in dates:
        start = date[0]
        end = date[1]
        #Comprehension likely goes here, or at least will affect the line below
        for row in twitter_df['date time']:
            row = parse(row)
            if row >= start and row <= end:
                #add point to new DB
                point_list.append(row) 
                #print(f'point found: {row}')
                
print(f'{len(point_list)} points found')

In [ ]:
with open('ovechkin_dates.csv', 'w', encoding='utf-8'):
    point_list = []
    twitter_handle = 'ovi8'
    tweet_sheet = f'./data/{twitter_handle}.csv'
    twitter_df = pd.read_csv(tweet_sheet, sep= '\t', header= 0, index_col= None)
    time = pd.date_range(start = (x), end= (y), freq = 'M')

    dates = find_dates('ovechal01')
    for date in dates:
        start = date[0]
        print('end', type(start))
        #start = parse(start)
        end = date[1]
        #end = parse(end)
        print('end', type(end))
        
        #Comprehension likely goes here, or at least will affect the line below
        
        for row in twitter_df['date time']:
            for r in twitter_df:
                print(row, type(row))
                try:
                    row = parse(row)
                    print(r)
                
                    if row >= start and row <= end:
                        #add point to new DB
                        if row in point_list:
                            break
                        else:
                            point_list.append(r) 
                            #print(f'point found: {row}')
                except:
                    continue
        
                
print(f'{len(point_list)} points found')

end <class 'datetime.datetime'>
end <class 'datetime.datetime'>
2021-12-02 18:19:51-05:00 <class 'str'>
date time
2021-12-02 18:19:51-05:00 <class 'datetime.datetime'>
2021-12-02 18:19:51-05:00 <class 'datetime.datetime'>
2021-11-20 20:44:54-05:00 <class 'str'>
date time
2021-11-20 20:44:54-05:00 <class 'datetime.datetime'>
2021-11-20 20:44:54-05:00 <class 'datetime.datetime'>
2021-11-16 04:04:50-05:00 <class 'str'>
date time
2021-11-16 04:04:50-05:00 <class 'datetime.datetime'>
2021-11-16 04:04:50-05:00 <class 'datetime.datetime'>
2021-11-04 13:16:26-04:00 <class 'str'>
date time
2021-11-04 13:16:26-04:00 <class 'datetime.datetime'>
2021-11-04 13:16:26-04:00 <class 'datetime.datetime'>
2021-10-28 22:15:31-04:00 <class 'str'>
date time
2021-10-28 22:15:31-04:00 <class 'datetime.datetime'>
2021-10-28 22:15:31-04:00 <class 'datetime.datetime'>
2021-10-12 22:41:50-04:00 <class 'str'>
date time
2021-10-12 22:41:50-04:00 <class 'datetime.datetime'>
2021-10-12 22:41:50-04:00 <class 'datetime

date time
2015-09-11 21:32:23-04:00 <class 'datetime.datetime'>
2015-09-11 21:32:23-04:00 <class 'datetime.datetime'>
2015-04-29 16:19:58-04:00 <class 'str'>
date time
2015-04-29 16:19:58-04:00 <class 'datetime.datetime'>
2015-04-29 16:19:58-04:00 <class 'datetime.datetime'>
2015-04-03 20:38:08-04:00 <class 'str'>
date time
2015-04-03 20:38:08-04:00 <class 'datetime.datetime'>
2015-04-03 20:38:08-04:00 <class 'datetime.datetime'>
2015-03-27 20:33:01-04:00 <class 'str'>
date time
2015-03-27 20:33:01-04:00 <class 'datetime.datetime'>
2015-03-27 20:33:01-04:00 <class 'datetime.datetime'>
2015-03-02 04:56:35-05:00 <class 'str'>
date time
2015-03-02 04:56:35-05:00 <class 'datetime.datetime'>
2015-03-02 04:56:35-05:00 <class 'datetime.datetime'>
2015-01-26 02:31:38-05:00 <class 'str'>
date time
2015-01-26 02:31:38-05:00 <class 'datetime.datetime'>
2015-01-26 02:31:38-05:00 <class 'datetime.datetime'>
2015-01-25 18:55:33-05:00 <class 'str'>
date time
2015-01-25 18:55:33-05:00 <class 'datetime

2014-06-22 09:32:44-04:00 <class 'str'>
date time
2014-06-22 09:32:44-04:00 <class 'datetime.datetime'>
2014-06-22 09:32:44-04:00 <class 'datetime.datetime'>
2014-06-22 09:19:13-04:00 <class 'str'>
date time
2014-06-22 09:19:13-04:00 <class 'datetime.datetime'>
2014-06-22 09:19:13-04:00 <class 'datetime.datetime'>
2014-06-22 05:21:17-04:00 <class 'str'>
date time
2014-06-22 05:21:17-04:00 <class 'datetime.datetime'>
2014-06-22 05:21:17-04:00 <class 'datetime.datetime'>
2014-06-21 19:37:38-04:00 <class 'str'>
date time
2014-06-21 19:37:38-04:00 <class 'datetime.datetime'>
2014-06-21 19:37:38-04:00 <class 'datetime.datetime'>
2014-06-21 10:02:50-04:00 <class 'str'>
date time
2014-06-21 10:02:50-04:00 <class 'datetime.datetime'>
2014-06-21 10:02:50-04:00 <class 'datetime.datetime'>
2014-06-21 07:52:31-04:00 <class 'str'>
date time
2014-06-21 07:52:31-04:00 <class 'datetime.datetime'>
2014-06-21 07:52:31-04:00 <class 'datetime.datetime'>
2014-06-20 12:18:17-04:00 <class 'str'>
date time
20

2013-09-29 16:13:16-04:00 <class 'datetime.datetime'>
2013-09-29 16:12:46-04:00 <class 'str'>
date time
2013-09-29 16:12:46-04:00 <class 'datetime.datetime'>
2013-09-29 16:12:46-04:00 <class 'datetime.datetime'>
2013-09-29 16:11:58-04:00 <class 'str'>
date time
2013-09-29 16:11:58-04:00 <class 'datetime.datetime'>
2013-09-29 16:11:58-04:00 <class 'datetime.datetime'>
2013-09-29 16:08:53-04:00 <class 'str'>
date time
2013-09-29 16:08:53-04:00 <class 'datetime.datetime'>
2013-09-29 16:08:53-04:00 <class 'datetime.datetime'>
2013-09-29 16:07:46-04:00 <class 'str'>
date time
2013-09-29 16:07:46-04:00 <class 'datetime.datetime'>
2013-09-29 16:07:46-04:00 <class 'datetime.datetime'>
2013-09-29 10:56:36-04:00 <class 'str'>
date time
2013-09-29 10:56:36-04:00 <class 'datetime.datetime'>
2013-09-29 10:56:36-04:00 <class 'datetime.datetime'>
2013-09-29 09:35:01-04:00 <class 'str'>
date time
2013-09-29 09:35:01-04:00 <class 'datetime.datetime'>
2013-09-29 09:35:01-04:00 <class 'datetime.datetime'

2013-04-07 15:00:57-04:00 <class 'datetime.datetime'>
2013-04-07 15:00:57-04:00 <class 'datetime.datetime'>
2013-04-05 23:35:23-04:00 <class 'str'>
date time
2013-04-05 23:35:23-04:00 <class 'datetime.datetime'>
2013-04-05 23:35:23-04:00 <class 'datetime.datetime'>
2013-04-05 23:34:04-04:00 <class 'str'>
date time
2013-04-05 23:34:04-04:00 <class 'datetime.datetime'>
2013-04-05 23:34:04-04:00 <class 'datetime.datetime'>
2013-04-04 15:48:15-04:00 <class 'str'>
date time
2013-04-04 15:48:15-04:00 <class 'datetime.datetime'>
2013-04-04 15:48:15-04:00 <class 'datetime.datetime'>
2013-04-03 21:14:43-04:00 <class 'str'>
date time
2013-04-03 21:14:43-04:00 <class 'datetime.datetime'>
2013-04-03 21:14:43-04:00 <class 'datetime.datetime'>
2013-04-01 18:12:34-04:00 <class 'str'>
date time
2013-04-01 18:12:34-04:00 <class 'datetime.datetime'>
2013-04-01 18:12:34-04:00 <class 'datetime.datetime'>
2013-03-31 03:18:19-04:00 <class 'str'>
date time
2013-03-31 03:18:19-04:00 <class 'datetime.datetime'

2011-09-18 01:26:02-04:00 <class 'str'>
date time
2011-09-18 01:26:02-04:00 <class 'datetime.datetime'>
2011-09-18 01:26:02-04:00 <class 'datetime.datetime'>
2011-09-17 17:10:53-04:00 <class 'str'>
date time
2011-09-17 17:10:53-04:00 <class 'datetime.datetime'>
2011-09-17 17:10:53-04:00 <class 'datetime.datetime'>
2011-09-12 13:06:47-04:00 <class 'str'>
date time
2011-09-12 13:06:47-04:00 <class 'datetime.datetime'>
2011-09-12 13:06:47-04:00 <class 'datetime.datetime'>
2011-09-09 20:10:38-04:00 <class 'str'>
date time
2011-09-09 20:10:38-04:00 <class 'datetime.datetime'>
2011-09-09 20:10:38-04:00 <class 'datetime.datetime'>
2011-09-09 18:24:14-04:00 <class 'str'>
date time
2011-09-09 18:24:14-04:00 <class 'datetime.datetime'>
2011-09-09 18:24:14-04:00 <class 'datetime.datetime'>
2011-09-09 03:47:07-04:00 <class 'str'>
date time
2011-09-09 03:47:07-04:00 <class 'datetime.datetime'>
2011-09-09 03:47:07-04:00 <class 'datetime.datetime'>
2011-09-09 02:33:05-04:00 <class 'str'>
date time
20

2009-01-30 20:11:16-05:00 <class 'datetime.datetime'>
2009-01-30 20:11:16-05:00 <class 'datetime.datetime'>
2009-01-30 20:07:38-05:00 <class 'str'>
date time
2009-01-30 20:07:38-05:00 <class 'datetime.datetime'>
2009-01-30 20:07:38-05:00 <class 'datetime.datetime'>
2009-01-30 20:06:56-05:00 <class 'str'>
date time
2009-01-30 20:06:56-05:00 <class 'datetime.datetime'>
2009-01-30 20:06:56-05:00 <class 'datetime.datetime'>
2009-01-30 20:05:54-05:00 <class 'str'>
date time
2009-01-30 20:05:54-05:00 <class 'datetime.datetime'>
2009-01-30 20:05:54-05:00 <class 'datetime.datetime'>
2009-01-30 20:03:52-05:00 <class 'str'>
date time
2009-01-30 20:03:52-05:00 <class 'datetime.datetime'>
2009-01-30 20:03:52-05:00 <class 'datetime.datetime'>
2009-01-30 20:02:59-05:00 <class 'str'>
date time
2009-01-30 20:02:59-05:00 <class 'datetime.datetime'>
2009-01-30 20:02:59-05:00 <class 'datetime.datetime'>
2009-01-30 20:02:39-05:00 <class 'str'>
date time
2009-01-30 20:02:39-05:00 <class 'datetime.datetime'

Will need to set up a comprehension here, to check tweet dates for datetimes already identified as valid points of data. As the list only currently contains the datetimes, without text

In [12]:
for point in point_list:
    print(point)

2013-03-07 17:22:54-05:00
2013-03-07 17:20:57-05:00
2013-03-14 02:57:16-04:00
2013-03-14 00:04:50-04:00
2013-03-14 00:00:46-04:00
2013-03-22 18:07:31-04:00
2013-03-22 18:04:13-04:00
2013-03-22 18:03:07-04:00
2013-03-24 12:54:47-04:00
2013-03-24 01:34:46-04:00
2013-03-24 01:30:36-04:00
2013-03-24 00:03:45-04:00
2013-03-23 19:03:04-04:00
2013-03-31 03:18:19-04:00
2013-04-04 15:48:15-04:00
2013-04-03 21:14:43-04:00
2013-04-05 23:35:23-04:00
2013-04-05 23:34:04-04:00
2013-04-07 15:00:57-04:00
2013-04-10 21:16:38-04:00
2013-04-10 20:39:59-04:00
2013-04-10 20:17:58-04:00
2013-04-17 19:46:44-04:00
2013-04-17 19:45:26-04:00
2013-04-17 19:44:16-04:00
2013-04-17 19:44:10-04:00
2013-04-24 19:12:31-04:00
2013-10-02 21:03:11-04:00
2013-10-02 20:58:44-04:00
2013-10-05 15:02:19-04:00
2013-11-09 02:17:13-05:00
2013-11-12 00:01:28-05:00
2013-11-27 00:55:33-05:00
2013-11-29 02:29:07-05:00
2013-12-23 00:37:46-05:00
2014-01-11 19:13:58-05:00
2014-02-08 03:19:33-05:00
2014-03-05 21:36:10-05:00
2014-03-15 2

In [ ]:
def creating_dates(row, date_list):
    dates = list(game_windows)
    print(dates)


In [33]:
game_windows

,start_date,end_date
0,2013-01-19 19:00:00-05:00,2013-01-18 19:00:00-05:00
1,2013-01-22 19:00:00-05:00,2013-01-21 19:00:00-05:00
2,2013-01-24 19:00:00-05:00,2013-01-23 19:00:00-05:00
3,2013-01-25 19:00:00-05:00,2013-01-24 19:00:00-05:00
4,2013-01-27 15:00:00-05:00,2013-01-26 15:00:00-05:00
...,...,...
675,2021-12-29 19:00:00-05:00,2021-12-28 19:00:00-05:00
676,2021-12-31 19:30:00-05:00,2021-12-30 19:30:00-05:00
677,2022-01-02 15:00:00-05:00,2022-01-01 15:00:00-05:00
678,2022-01-07 20:00:00-05:00,2022-01-06 20:00:00-05:00


Not catching data currently, date ranges need to be defined as demonstrated above

In [19]:
from datetime import timedelta

data_points = []
post_game = 0
tweet_dates = twitter_df.iloc[:,0]
#tweet_times = twitter_df.iloc[:,1]
for dt in tweet_dates:
    dt = pd.to_datetime(dt)
    #Use timedates to create if statements
    #Try within 24 hours of puckdrop, also from midnight on gameday
    if dt in game_list:
        gm_time = pd.to_datetime(gm_time)
        gm = pd.to_datetime(gm)
        if gm < gm_time:
            data_points.append(gm)
        else:
            post_game += 1


print(f'{len(data_points)} good points of data, {post_game} postgame tweets')

0 good points of data, 0 postgame tweets


In [9]:
for d in data_points:
    print(d)

Maybe need to include puck drop times? Would require altering stat scraper, and likely slowing it down signifigantly, to the magnitute of rendering an entire new webpage in beautfiul soup per row in the dataset. For Ovechkin this would include 1228 pages! Maybe I should use selenium as an alternative. However at least for Ovechkin, no gameday tweets were scheduled for 7pm EST. BRB Looking at tweet timestamps

In [9]:
for point in data_points:
    print(point)